In [ ]:
%load_ext nb_black

# ⚾ PITCHf/x ⚾

PITCHf/x is a system developed by Sportvision and introduced in Major League Baseball (MLB) during the 2006 playoffs. It uses two cameras to record the position of the pitched baseball during its flight from the pitcher’s hand to home plate, and various parameters are measured and calculated to describe the trajectory and speed of each pitch. It is now instituted in all ballparks in MLB.

This version of the data used was downloaded from the `.Rdata` file provided on [this page](https://www2.stat.duke.edu/courses/Summer17/sta101.001-2/uploads/project/project.html) (direct link to [`.Rdata` file](http://stat.duke.edu/courses/Summer17/sta101.001-2/uploads/project/mondayBaseball.Rdata)).  There is also a full data dictionary on [this page](https://www2.stat.duke.edu/courses/Summer17/sta101.001-2/uploads/project/project.html).  There are many other ways to gather the data as shown in ['A new Python-based PITCHf/x parser & scraper'](https://www.beyondtheboxscore.com/2015/9/24/9374949/a-new-python-based-pitchf-x-parser-scraper) by John Choiniere.

We'll specifically be looking into predicting `pitchType`.

## EDA and Data Cleaning

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from mlxtend.plotting import plot_decision_regions
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline


data_url = "https://docs.google.com/spreadsheets/d/1pmBtSw7v_tU_dIX1-4E8_Q7wC43fDs6LGDQzN49-ffk/export?format=csv"
pitch = pd.read_csv(data_url)

Do some initial EDA to get familiar with what data we have

We have a lot of data.  We're going to take the easy way out and drop all the missing values...

In [ ]:
pitch = pitch.dropna()

There's a lot of id type columns that aren't too useful for general predictions of `pitchType`.  Sure, what pitcher is throwing the ball would probably be predictive, but we want our model to work years down the road when these pitchers might not be playing anymore.  If we were wanting to update our model very often, then including pitcher might be a good feature.

Drop the id columns from our dataset

In [ ]:
id_cols = [
    "gameString",
    "gameDate",
    "visitor",
    "home",
    "batterId",
    "batterName",
    "pitcherId",
    "pitcherName",
    "catcherId",
    "catcher",
    "umpireId",
    "umpire",
]

pitch = _____

We now have essentially 3 classes of columns now:

* Before pitch - stats about batter handedness etc.
* After pitch - metrics about pitch speed etc.
* After hit - metrics about what angle the ball was hit at etc.

Let's say we're making our predictions for a broadcasting company.  Our broadcast wants to automatically display on the screen what the pitch was based on metrics we get instantaneously about the pitch's speed/movement.

Based on this, we wouldn't want to include metrics that happen after a hit. Drop the after hit columns from our dataset.

In [ ]:
before_pitch_cols = [
    "inning",
    "side",
    "balls",
    "strikes",
    "outs",
    "batterHand",
    "batterPosition",
    "pitcherHand",
    "timesFaced",
]

after_pitch_cols = [
    "releaseVelocity",
    "spinRate",
    "spinDir",
    "locationHoriz",
    "locationVert",
    "movementHoriz",
    "movementVert",
]

after_hit_cols = [
    "probCalledStrike",
    "battedBallType",
    "battedBallAngle",
    "battedBallDistance",
    "paResult",
    "pitchResult",
]

pitch = pitch.drop(columns=after_hit_cols)

Look at the `head` of the dataframe again, we cutout a lot of columns so this is a much more managable view of the data.

In [ ]:
pitch.head()

We want to predict the `pitchType`.  A dictionary has been created below to translate the abbreviations into a longer/more readable name.

In [ ]:
pitch_abbr = {
    "FT": "fastball (two-seam)",
    "FF": "fastball (four-seam)",
    "FC": "fastball (cutter)",
    "FS": "fastball (splitter)",
    "SI": "sinker",
    "SL": "slider",
    "CU": "curveball",
    "KC": "knuckle-curve",
    "EP": "eephus",
    "CH": "changeup",
    "SC": "screwball",
    "KN": "knuckleball",
    "UN": "unidentified",
}

* Translate the `pitchType` column into its longer form names
* View the counts of each pitch.  
    * Instead of just printing the pitch counts.  Create a barplot of these counts.

Let's focus only on predicting some of the major classes of pitches.  Let's filter down to an arbitray list of 3 (`pitches_we_care_about`).

* Use the predefined list to filter our dataframe to only `pitchType`s in the list

In [ ]:
pitches_we_care_about = ["fastball (four-seam)", "changeup", "curveball"]

View the `shape` and `columns` to get a feel for how much data we removed from the problem.

In [ ]:
pitch.shape

In [ ]:
pitch.columns

Make a scatter plot of `balls` x `strikes` colored by `pitchType`.  What might be a better way to view this data?

Make 2 separate barplots (1 for `balls` and 1 for `strikes`) colored by `pitchType`.

The main thing I take away from these plots is that a four-seam fastball is the favorite pitch no matter what the count is.

* Make a scatterplot of `releaseVelocity` x `spinDir` colored by `pitchType`.
* Plot a horizontal line when `spinDir == 180`.
* Add transparency to the plot so you can see if an area of the plot is denser or if its just a couple of points.

We see a mirror image in this plot... What could explain this and how might we take action on it?

* Create a new version of spin direction that is a pitch's absolute difference from 180.
* Call it `spinMag` for spin magnitude.
* Drop `spinDir` from the dataframe
* Redo the plot with `spinRate` x `spinMag`.

This data looks linearly separable already... to have a simpler model we might just proceed with these columns only.

Advantages of this

1. It's simple to explain rather than bringing in all these additional features that are essentially dead weight if we find out we do get good predictions with just these variables.
* We can easily plot the decision boundaries if we only have 2 inputs.

## Model Prep

* Split the data into `X` and `y`
* Only use `spinMag` and `spinRate` as predictors

In [ ]:
X = _____
y = pitch["pitchType"]

Convert y to an integer with `[0, 1, 2]` representing each of our classes.  Technically, this model can handle strings as labels, but many other machine learning tools prefer integer. So it's good to practice this step.  `sklearn` provides the [`LabelEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) for this task, and you can research this on your own (and we'll use it later), but for today let's do a more manual/pure python way of doing this.

* Create a dictionary of replacement values
* `print` out the y `value_counts` before replacing to ensure we don't make a mistake
* Use `replace` to convert the values to our `int` labels
* `print` out the y `value_counts` after replacing to ensure we did't make a mistake
* `print` out the dictionary to show a legend for the new labels

* Perform a train/test split
    * Use 20% of the data for testing
    * Stratify the split by the class labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## Model Build

* Fit a Support Vector Classifier (`SVC`)
    * Use the `'linear'` `kernel`.
    * Use 1.0 for the value of `C`

## Model Evaluation

Score the model for both your train and test set to diagnose under/overfitting.

In [ ]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score:.2f}")
print(f"Test score: {test_score:.2f}")

* Show a confusion matrix.
   * Which classes are most likely to be confused?
* Show a classification report.
   * What pitch type do we classify with the highest precision? What does that mean?

In [ ]:
# 0 - fastball
# 1 - changeup
# 2 - curveball


Plot the decision boundary of the model in relation to our spin predictors.

In [ ]:
X_np = X_train.values
y_np = y_train.values

plot_decision_regions(X_np, y_np, clf=model, scatter_kwargs={"alpha": 0.1})
plt.xlabel("spinRate")
plt.ylabel("spinMag")
plt.show()

## Hyperparameter Optimization

The main hyperparameters we toy with when fitting a `SVC` is the `kernel` and the value of `C`.  The below quotes are from [this article](https://medium.com/all-things-ai/in-depth-parameter-tuning-for-svc-758215394769) which gives a pretty good overview of both of these hyperparameters. Additionally, the article discusses the `degree` (relevant when `kernel` is `'poly'`) and `gamma` (relevant when `kernel` is not `'linear'`).

> C is the penalty parameter of the error term. It controls the trade off between smooth decision boundary and classifying the training points correctly.

> Increasing C values may lead to overfitting the training data.

When the `kernel` is `'linear'` we won't see a huge effect when varying the value of `C`.  Its effects are more visible in the decision boundary when we use a non-linear kernel.  Because of this, we're going to change the kernel to `'rbf'` without an explanation of what the means at this point...

In [ ]:
X_np = X_train.values
y_np = y_train.values


cs = [0.1, 1, 10, 100, 1000, 10000]
# The above list can be created fancily with the below
# Read as: "go from 10^(-1) to 10^4 in 6 steps"
cs = np.logspace(-1, 4, 6)
for c in cs:
    model = SVC(kernel="rbf", C=c)
    model.fit(X_train, y_train)

    plot_decision_regions(X_np, y_np, clf=model, scatter_kwargs={"alpha": 0.1})
    plt.title(f"C={c}")
    plt.show()

## Aside about multiclass in `SVC`

SVMs can only draw a single decision boundary, which means that they can only separate 2 classes.  So what is `sklearn` doing that allows us to do this 3 class problem?  From [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) we see the below description for the `decision_function_shape` parameter.

> **decision_function_shape** `‘ovo’`, `‘ovr’`, `default=’ovr’`
>    
> Whether to return a one-vs-rest (`‘ovr’`) decision function of shape (n_samples, n_classes) as all other classifiers, or the original one-vs-one (`‘ovo’`) decision function of libsvm which has shape (n_samples, n_classes * (n_classes - 1) / 2). However, one-vs-one (`‘ovo’`) is always used as multi-class strategy.

### One-vs-One (ovo)

In ovo, we just consider 2 classes at a time and then do some math to combine these multiple decision boundaries.

For example, in this data we would fit 3 models:
* curve vs fastball
* curve vs changeup
* fastball vs changeup

Each of these are then used to make predictions, and we end up with essentially a majority vote like in KNN (this is over simplified but this as a mental model is ok).

### One-vs-Rest (ovr)

In ovr, we'll just look at one of our classes as the class of interest at a time and build a model for each class.

For example, in this data we would fit 3 models:
* curve vs not-curve
* fastball vs not-fastball
* changeup vs not-changeup

Again, we'll use these sub-models to essentially vote.  This is where the analogy to voting can fall apart (what if every model predicts the 'not' class? This where some probabilities come into play).